# CMPT743 Lab 1

### Question 1 (20 mins)

Use PyTorch to find the numerical solution to $2x^2-4x+1$

In [3]:
import torch

def equation(x):
    return 2*x**2 - 4*x + 1

# initialize x with a random value
x = torch.randn(1, requires_grad=True)

# set up the optimizer, here we use stochastic gradient descent
optimizer = torch.optim.SGD([x], lr=0.01)

# optimization loop
for step in range(1000):
    optimizer.zero_grad()
    loss = equation(x)
    loss.backward()
    optimizer.step()

    if step % 100 == 0:
        print(f"step {step}, x = {x.item()}, equation value = {loss.item()}")

# result
print(f"Root approximation: x = {x.item()}")


step 0, x = -0.5330286622047424, equation value = 4.100210189819336
step 100, x = 0.9741373062133789, equation value = -0.9985483884811401
step 200, x = 0.9995637536048889, equation value = -0.9999996423721313
step 300, x = 0.9999926090240479, equation value = -1.0
step 400, x = 0.9999992847442627, equation value = -1.0
step 500, x = 0.9999992847442627, equation value = -1.0
step 600, x = 0.9999992847442627, equation value = -1.0
step 700, x = 0.9999992847442627, equation value = -1.0
step 800, x = 0.9999992847442627, equation value = -1.0
step 900, x = 0.9999992847442627, equation value = -1.0
Root approximation: x = 0.9999992847442627


### Question 2 (20 mins)

Implement a custom activation function $f(x) = ln(1 + e^x)$, and integrate it into a simple neural network to approximate a simple function.

In [17]:
import torch
import torch.nn as nn
import numpy as np

# Custom Activation Function
class CustomActivation(nn.Module):
    def __init__(self):
        super(CustomActivation, self).__init__()

    def forward(self, x):
        return torch.log(1 + torch.exp(x))


In [15]:
# Define a Simple Neural Network with the Custom Activation Function
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(1, 10)  # Input layer
        self.fc2 = nn.Linear(10, 10) # Hidden layer
        self.fc3 = nn.Linear(10, 1)  # Output layer
        self.custom_act = CustomActivation()

    def forward(self, x):
        x = self.custom_act(self.fc1(x))
        x = self.custom_act(self.fc2(x))
        return self.fc3(x)

# Create the neural network
model = SimpleNN()

# Define Loss Function and Optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


In [6]:
# Training Data for y = sin(x)
x_train = torch.linspace(-10, 10, 1000).view(-1, 1)
y_train = torch.sin(x_train)

# Training Loop
epochs = 1000
for epoch in range(epochs):
    optimizer.zero_grad()
    output = model(x_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch [{epoch}/{epochs}], Loss: {loss.item()}')

# Test the model
with torch.no_grad():
    y_pred = model(x_train)


Epoch [0/1000], Loss: 0.6958268284797668
Epoch [100/1000], Loss: 0.44919657707214355
Epoch [200/1000], Loss: 0.3510535657405853
Epoch [300/1000], Loss: 0.1815216988325119
Epoch [400/1000], Loss: 0.09083357453346252
Epoch [500/1000], Loss: 0.022584060207009315
Epoch [600/1000], Loss: 0.004558686167001724
Epoch [700/1000], Loss: 0.002822058042511344
Epoch [800/1000], Loss: 0.0020239190198481083
Epoch [900/1000], Loss: 0.0015511605888605118


In [10]:
print(f'y_train [{y_train[0:5]}], \nperdiction: {y_pred[0:5]}')

y_train [tensor([[0.5440],
        [0.5271],
        [0.5100],
        [0.4927],
        [0.4752]])], 
perdiction: tensor([[0.5846],
        [0.5623],
        [0.5400],
        [0.5178],
        [0.4956]])


### Question 3 (10 mins)

Use ONNX/Netron tools to visualize resnet18 architecture from torchvision library

In [12]:
import torch
import torchvision

# Load pretrained ResNet18 model
model = torchvision.models.resnet18(pretrained=True)

# Set the model to evaluation mode
model.eval()

# Create a dummy input tensor
dummy_input = torch.randn(1, 3, 224, 224)

# Export the model
torch.onnx.export(model, 
                  dummy_input, 
                  "resnet18.onnx", 
                  export_params=True, 
                  opset_version=10, 
                  do_constant_folding=True, 
                  input_names=['input'], 
                  output_names=['output'])


c:\Users\Alienware\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\Users\Alienware\AppData\Local\Programs\Python\Python39\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
c:\Users\Alienware\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Alienware\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arg

### Question 4 (30 mins)

Design a neural network in PyTorch that has two parallel input branches, combines these inputs with additional data midway through the network, and then splits into two separate output branches.

1. **Input Layer**:
    - The network starts with two parallel input branches.
    - Each branch should accept an input tensor of shape **`(N, 10)`**, where **`N`** is the batch size.
2. **First and Second Branch**:
    - **Branch 1 and Branch 2** are identical in structure.
    - Each branch consists of the following layers:
        - A linear layer that expands the input from 10 to 20 features.
        - A ReLU activation layer.
        - Another linear layer that further expands from 20 to 30 features.
3. **Midway Additional Inputs**:
    - After the first and second branches, introduce an additional input tensor of shape **`(N, 5)`**.
    - This additional input represents extra features to be combined with the outputs of the two branches.
4. **Combination of Branch Outputs and Additional Input**:
    - Concatenate the outputs of the two branches (each of shape **`(N, 30)`**) with the additional input (shape **`(N, 5)`**), resulting in a tensor of shape **`(N, 65)`**.
5. **Shared Layers After Combination**:
    - Pass the combined tensor through a shared linear layer that reduces the dimension from 65 to 50.
    - Apply a ReLU activation function.
6. **Two Separate Output Branches**:
    - Split into two separate output branches after the shared layers.
    - **Output Branch 1** and **Output Branch 2**:
        - Each output branch consists of a single linear layer that maps the 50 features to a single output feature (shape **`(N, 1)`**).

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Custom Neural Network
class CustomNetwork(nn.Module):
    def __init__(self):
        super(CustomNetwork, self).__init__()

        # Branch layers (shared between both branches)
        self.branch_fc1 = nn.Linear(10, 20)
        self.branch_fc2 = nn.Linear(20, 30)

        # Shared layers afte r combining the branches
        self.shared_fc1 = nn.Linear(65, 50)

        # Separate output branches
        self.output_branch1 = nn.Linear(50, 1)
        self.output_branch2 = nn.Linear(50, 1)

    def forward(self, x1, x2, additional_input):
        # First and second branches
        x1 = F.relu(self.branch_fc1(x1))
        x1 = self.branch_fc2(x1)

        x2 = F.relu(self.branch_fc1(x2))
        x2 = self.branch_fc2(x2)

        # Combine the outputs of the two branches with the additional input
        combined = torch.cat((x1, x2, additional_input), dim=1)

        # Shared layers after combination
        combined = F.relu(self.shared_fc1(combined))

        # Two separate output branches
        output1 = self.output_branch1(combined)
        output2 = self.output_branch2(combined)

        return output1, output2

# Create the neural network
model = CustomNetwork()

# Example usage
N = 5  # Example batch size
input1 = torch.randn(N, 10)
input2 = torch.randn(N, 10)
additional_input = torch.randn(N, 5)
output1, output2 = model(input1, input2, additional_input)

print("Output 1:", output1)
print("Output 2:", output2)




Output 1: tensor([[-0.0299],
        [-0.0179],
        [-0.0395],
        [ 0.0819],
        [-0.0777]], grad_fn=<AddmmBackward0>)
Output 2: tensor([[ 0.1179],
        [ 0.1077],
        [ 0.0515],
        [-0.0801],
        [ 0.2045]], grad_fn=<AddmmBackward0>)
